In [66]:
import sqlite3
from sklearn.datasets import fetch_lfw_people
import pandas as pd

# load the data using sklearn
lfw_dataset = fetch_lfw_people(data_home='/Users/shahd.metwally/LFW/', min_faces_per_person=1, download_if_missing=False)
df = pd.DataFrame(data=lfw_dataset.data)

# connecting to our SQLite database
conn = sqlite3.connect('lfw_dataset.db')
cursor = conn.cursor()

# creating a table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS faces (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL
    )
''')
cursor.execute('DELETE FROM faces')
# and adding the names to the table
for target_name in lfw_dataset.target_names:
    cursor.execute("INSERT INTO faces (name) VALUES (?)", (target_name,))

# commit the changes and close the connection
conn.commit()
conn.close()

# printing the rows in the dataset to see if it works fine (just for testing)
conn = sqlite3.connect('lfw_dataset.db')
cursor = conn.cursor()
    
cursor.execute('SELECT * FROM faces')

rows = cursor.fetchall()

if not rows:
    print("No data.")
else:
    print(f"Total rows: {len(rows)}")
    # Print the data
    for row in rows:
        print(f"ID: {row[0]}, Name: {row[1]}")

# Close the connection
conn.close()


Total rows: 5749
ID: 1, Name: AJ Cook
ID: 2, Name: AJ Lamas
ID: 3, Name: Aaron Eckhart
ID: 4, Name: Aaron Guiel
ID: 5, Name: Aaron Patterson
ID: 6, Name: Aaron Peirsol
ID: 7, Name: Aaron Pena
ID: 8, Name: Aaron Sorkin
ID: 9, Name: Aaron Tippin
ID: 10, Name: Abba Eban
ID: 11, Name: Abbas Kiarostami
ID: 12, Name: Abdel Aziz Al-Hakim
ID: 13, Name: Abdel Madi Shabneh
ID: 14, Name: Abdel Nasser Assidi
ID: 15, Name: Abdoulaye Wade
ID: 16, Name: Abdul Majeed Shobokshi
ID: 17, Name: Abdul Rahman
ID: 18, Name: Abdulaziz Kamilov
ID: 19, Name: Abdullah
ID: 20, Name: Abdullah Ahmad Badawi
ID: 21, Name: Abdullah Gul
ID: 22, Name: Abdullah Nasseef
ID: 23, Name: Abdullah al-Attiyah
ID: 24, Name: Abdullatif Sener
ID: 25, Name: Abel Aguilar
ID: 26, Name: Abel Pacheco
ID: 27, Name: Abid Hamid Mahmud Al-Tikriti
ID: 28, Name: Abner Martinez
ID: 29, Name: Abraham Foxman
ID: 30, Name: Aby Har-Even
ID: 31, Name: Adam Ant
ID: 32, Name: Adam Freier
ID: 33, Name: Adam Herbert
ID: 34, Name: Adam Kennedy
ID: 35, 